In [3]:
from dna_features_viewer import BiopythonTranslator
import matplotlib.pyplot as plt
import bokeh
from bokeh import 

graphic_record = BiopythonTranslator().translate_record("./gene_neighborhood.gff")

p = graphic_record.plot_with_bokeh(figure_width=10)
show(p)
#ax.figure.tight_layout()

NameError: name 'show' is not defined

In [6]:
import dna_features_viewer as dfv
import bokeh
import bokeh.plotting
bokeh.io.output_notebook()

features = [
    dfv.GraphicFeature(
        start=5, end=20, strand=+1, color="#ffd700", label="Small feature"
    ),
    dfv.GraphicFeature(
        start=20,
        end=500,
        strand=+1,
        color="#ffcccc",
        label="Gene 1 with a very long name",
    ),
    dfv.GraphicFeature(
        start=400, end=700, strand=-1, color="#cffccc", label="Gene 2"
    ),
    dfv.GraphicFeature(
        start=600, end=900, strand=+1, color="#ccccff", label="Gene 3"
    )
]
for f in features:
    f.linecolor='green'
    f.box_color='orange' # box color is not being changed??
    f.label_link_color='#ccccff' # label link color is note being changed??

record = dfv.GraphicRecord(sequence_length=1000, features=features)
record.plot()

#bokeh.plotting.show() # bokeh figure is not being output to notebook because it is formatted as html doc

Loading BokehJS ...

(<Axes: >,
 ({GF(Small feature, 5-20 (1)): 0,
   GF(Gene 1 with a very long name, 20-500 (1)): 0,
   GF(Gene 2, 400-700 (-1)): 1.0,
   GF(Gene 3, 600-900 (1)): 0},
  {GF(Small feature, 5-20 (1)): {'feature_y': 0, 'annotation_y': 1.0}}))

In [26]:
import dna_features_viewer as dfv
import bokeh
from bokeh.plotting import figure, show, save
from bokeh.io import output_notebook, export_png
import matplotlib.pyplot as plt

features = [
    dfv.GraphicFeature(
        start=5, end=20, strand=+1, color="#ffd700", label="Small feature"
    ),
    dfv.GraphicFeature(
        start=20,
        end=500,
        strand=+1,
        color="#ffcccc",
        label="Gene 1 with a very long name",
    ),
    dfv.GraphicFeature(start=400, end=700, strand=-1, color="#cffccc", label="Gene 2"),
    dfv.GraphicFeature(start=600, end=900, strand=+1, color="#ccccff", label="Gene 3"),
]
for f in features:
    f.linecolor = "green"
    f.box_color = "orange"  # box color is not being changed??
    f.label_link_color = "#ccccff"  # label link color is note being changed??


output_notebook()

record = dfv.GraphicRecord(sequence_length=1000, features=features)
fig = record.plot_with_bokeh()

show(fig)

#show(record)

#export_png(record, filename="plot.png")
# bokeh.plotting # bokeh figure is not being output to notebook because it is formatted as html doc



Loading BokehJS ...

In [30]:
import dna_features_viewer as dfv
import bokeh
from bokeh.plotting import figure, show, save
from bokeh.io import output_notebook, export_png

record = dfv.BiopythonTranslator().translate_record("./nife_test.gff")

record = record.crop((200000, 220000))

output_notebook()

#record = record.dfv.GraphicRecord()
fig = record.plot_with_bokeh()

show(fig)


Loading BokehJS ...

In [31]:
import dna_features_viewer as dfv
from bokeh.plotting import show, output_notebook

output_notebook()

# Load the full record
record = dfv.BiopythonTranslator().translate_record("nife_test.gff")

# Define crop region
crop_start = 200000
crop_end = 220000

# Manually crop features and adjust coordinates
cropped_features = [
    f.copy().crop((crop_start, crop_end)) for f in record.features
    if f.end >= crop_start and f.start <= crop_end
]

# Adjust feature coordinates to new origin (start at 0)
for f in cropped_features:
    f.start -= crop_start
    f.end -= crop_start

# Create a new GraphicRecord
cropped_record = dfv.GraphicRecord(
    sequence_length=(crop_end - crop_start),
    features=cropped_features
)

# Plot with Bokeh
fig = cropped_record.plot_with_bokeh()
show(fig)


Loading BokehJS ...

AttributeError: 'GraphicFeature' object has no attribute 'copy'

In [33]:
import dna_features_viewer as dfv
from bokeh.plotting import show, output_notebook

output_notebook()

# Load the full record
record = dfv.BiopythonTranslator().translate_record("nife_test.gff")

# Define crop region
crop_start = 200000
crop_end = 220000

# Manually select and translate features that overlap the window
cropped_features = []
for f in record.features:
    if f.end >= crop_start and f.start <= crop_end:
        new_start = max(f.start, crop_start) - crop_start
        new_end = min(f.end, crop_end) - crop_start
        cropped_feature = dfv.GraphicFeature(
            start=new_start,
            end=new_end,
            strand=f.strand,
            color=f.color,
            label=f.label,
            label_link_color=f.label_link_color,
            fontdict=f.fontdict,
            linewidth=f.linewidth,
            linecolor=f.linecolor
        )
        cropped_features.append(cropped_feature)

# Create new GraphicRecord for the cropped region
cropped_record = dfv.GraphicRecord(
    sequence_length=crop_end - crop_start,
    features=cropped_features
)

# Plot with Bokeh
fig = cropped_record.plot_with_bokeh()
show(fig)


Loading BokehJS ...

In [21]:
from bokeh.plotting import figure, output_file, show

# create a figure object
p = figure(width=300, height=300, tools="pan,reset,save")

# add a Circle renderer to this figure
p.circle([1, 2.5, 3, 2], [2, 3, 1, 1.5], radius=0.3, alpha=0.5)

# specify how to output the plot(s)
output_file("foo.html")

# display the figure
show(p)


In [37]:
import pandas as pd
import dna_features_viewer as dfv
from bokeh.plotting import show, output_notebook
from bokeh.models import Range1d

# --- Configuration ---
gff_file = "nife_test.gff"
gene_name = "AMXMAG_0088___3611"  # <-- Replace with your gene name
window = 10000  # Window size upstream/downstream

output_notebook()

# --- Step 1: Locate gene coordinates from GFF ---
gff = pd.read_csv(
    gff_file,
    sep="\t",
    comment="#",
    header=None,
    names=[
        "seqid",
        "source",
        "type",
        "start",
        "end",
        "score",
        "strand",
        "phase",
        "attributes",
    ],
)

# Find the gene by Name= or ID= (adjust as needed)
gene_row = gff[
    gff["attributes"].str.contains(f"Name={gene_name}|ID={gene_name}", na=False)
]

if gene_row.empty:
    raise ValueError(f"Gene '{gene_name}' not found in {gff_file}")

gene_start = gene_row.iloc[0]["start"]
gene_end = gene_row.iloc[0]["end"]
crop_start = max(0, gene_start - window)
crop_end = gene_end + window

# --- Step 2: Translate to GraphicRecord ---
record = dfv.BiopythonTranslator().translate_record(gff_file)

# --- Step 3: Manually crop and adjust features ---
cropped_features = []
for f in record.features:
    if f.end >= crop_start and f.start <= crop_end:
        new_start = max(f.start, crop_start) - crop_start
        new_end = min(f.end, crop_end) - crop_start
        new_feature = dfv.GraphicFeature(
            start=new_start,
            end=new_end,
            strand=f.strand,
            color=f.color,
            label=f.label,
            label_link_color=f.label_link_color,
            fontdict=f.fontdict,
            linewidth=f.linewidth,
            linecolor=f.linecolor,
        )
        cropped_features.append(new_feature)

# --- Step 4: Create cropped record and plot ---
cropped_record = dfv.GraphicRecord(
    sequence_length=crop_end - crop_start, features=cropped_features
)

fig = cropped_record.plot_with_bokeh()

# --- Step 5: Adjust Bokeh axis to show original genomic coordinates ---
fig.x_range = Range1d(start=crop_start, end=crop_end)
fig.xaxis.axis_label = "Genomic Position (bp)"
fig.xaxis[0].formatter.use_scientific = False  # Optional: avoid sci notation

# Optional: re-adjust label positions if they overlap due to new scale

# --- Step 6: Show the plot ---
show(fig)


Loading BokehJS ...